In [1]:
# Cell 1 — install (safe to run)
import sys
!"{sys.executable}" -m pip install --upgrade pip
!"{sys.executable}" -m pip install openpyxl pandas sqlalchemy psycopg2-binary


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1


In [2]:
# Cell 2 — load the Excel file
import pandas as pd

df = pd.read_excel("customer_shopping_behavior_analysis.xlsx")

print("Rows, Cols:", df.shape)
df.head()


Rows, Cols: (3900, 18)


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [3]:
# Cell 3 — Clean column names (snake_case)

# Make all column names lowercase
df.columns = df.columns.str.lower()

# Replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_')

# Fix specific column names if needed
df = df.rename(columns={
    'purchase_amount_(usd)': 'purchase_amount'  # your file uses this format
})

df.head()


,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [4]:
# Cell 4 — Quick EDA: structure, stats, and missing values
print("=== df.info() ===")
df.info()

print("\n=== df.describe() (numeric) ===")
display(df.describe(include=[float, int]))

print("\n=== df.describe(include='all') (all columns) ===")
display(df.describe(include='all'))

print("\n=== Missing values per column ===")
missing = df.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])   # only show columns with missing values


=== df.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             3900 non-null   int64  
 1   age                     3900 non-null   int64  
 2   gender                  3900 non-null   object 
 3   item_purchased          3900 non-null   object 
 4   category                3900 non-null   object 
 5   purchase_amount         3900 non-null   int64  
 6   location                3900 non-null   object 
 7   size                    3900 non-null   object 
 8   color                   3900 non-null   object 
 9   season                  3900 non-null   object 
 10  review_rating           3863 non-null   float64
 11  subscription_status     3900 non-null   object 
 12  shipping_type           3900 non-null   object 
 13  discount_applied        3900 non-null   object 
 14  promo_code_used       

,customer_id,age,purchase_amount,review_rating,previous_purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000



=== df.describe(include='all') (all columns) ===


,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
count,3900.000000,3900.000000,3900,3900,3900,3900.000000,3900,3900,3900,3900,3863.000000,3900,3900,3900,3900,3900.000000,3900,3900
unique,NaN,NaN,2,25,4,NaN,50,4,25,4,NaN,2,6,2,2,NaN,6,7
top,NaN,NaN,Male,Blouse,Clothing,NaN,Montana,M,Olive,Spring,NaN,No,Free Shipping,No,No,NaN,PayPal,Every 3 Months
freq,NaN,NaN,2652,171,1737,NaN,96,1755,177,999,NaN,2847,675,2223,2223,NaN,677,584
mean,1950.500000,44.068462,NaN,NaN,NaN,59.764359,NaN,NaN,NaN,NaN,3.750065,NaN,NaN,NaN,NaN,25.351538,NaN,NaN
std,1125.977353,15.207589,NaN,NaN,NaN,23.685392,NaN,NaN,NaN,NaN,0.716983,NaN,NaN,NaN,NaN,14.447125,NaN,NaN
min,1.000000,18.000000,NaN,NaN,NaN,20.000000,NaN,NaN,NaN,NaN,2.500000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
25%,975.750000,31.000000,NaN,NaN,NaN,39.000000,NaN,NaN,NaN,NaN,3.100000,NaN,NaN,NaN,NaN,13.000000,NaN,NaN
50%,1950.500000,44.000000,NaN,NaN,NaN,60.000000,NaN,NaN,NaN,NaN,3.800000,NaN,NaN,NaN,NaN,25.000000,NaN,NaN
75%,2925.250000,57.000000,NaN,NaN,NaN,81.000000,NaN,NaN,NaN,NaN,4.400000,NaN,NaN,NaN,NaN,38.000000,NaN,NaN



=== Missing values per column ===
review_rating    37
dtype: int64


In [5]:
# --- Renaming columns to snake_case for PostgreSQL compatibility ---

df.columns = (
    df.columns
    .str.strip()                # remove extra spaces
    .str.lower()                # convert to lowercase
    .str.replace(' ', '_')      # replace spaces with underscore
    .str.replace('(', '', regex=False)  # remove (
    .str.replace(')', '', regex=False)  # remove )
)

# Show new column names
df.columns


Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [6]:
df['review_rating'] = df.groupby('category')['review_rating'] \
                        .transform(lambda x: x.fillna(x.median()))


In [7]:
# Safe group-wise median imputation for review_rating
import pandas as pd

# quick checks (will raise helpful message if column names differ)
if 'review_rating' not in df.columns:
    raise KeyError("Column 'review_rating' not found. Run print(df.columns) and check names.")
if 'category' not in df.columns:
    raise KeyError("Column 'category' not found. Run print(df.columns) and check names.")

# convert to numeric just in case (non-numeric -> NaN)
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

# group-wise median fill
df['review_rating'] = df.groupby('category')['review_rating'].transform(lambda x: x.fillna(x.median()))

# Report results
print("Missing review_rating after fill:", df['review_rating'].isnull().sum())
print("\nreview_rating summary after imputation:")
print(df['review_rating'].describe())

# show a few rows to verify
df[['category','review_rating']].sample(8)


Missing review_rating after fill: 0

review_rating summary after imputation:
count    3900.000000
mean        3.750051
std         0.713590
min         2.500000
25%         3.100000
50%         3.800000
75%         4.400000
max         5.000000
Name: review_rating, dtype: float64


,category,review_rating
1944,Clothing,2.7
17,Clothing,4.7
2513,Accessories,3.5
1418,Clothing,4.1
3262,Clothing,2.7
2240,Footwear,4.3
1997,Accessories,4.7
902,Accessories,2.8


## Connecting Python script to PostgreSQL

In [9]:
from sqlalchemy import create_engine

# PostgreSQL login details
username = "postgres"        # same as pgAdmin
password = "915046"          # YOUR password
host = "localhost"
port = "5432"
database = "customer_behaviour"   # your DB name in pgAdmin

# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
)

print("Connected to PostgreSQL!")


Connected to PostgreSQL!


In [10]:
table_name = "customer"

df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data uploaded to table '{table_name}' successfully!")


Data uploaded to table 'customer' successfully!
